In [ ]:
def set_latex():
    for i in range(2):
        import matplotlib
        import matplotlib.pyplot as plt

        plt.rc('text', usetex=True)
        plt.rc('font', family='serif')

        plt.style.use("default")
        plt.rcParams["font.size"] = 15

        plt.rcParams['font.family'] = 'Times New Roman'
        plt.rcParams['mathtext.fontset'] = 'stix'

        try:
            del matplotlib.font_manager.weight_dict['roman']
            matplotlib.font_manager._rebuild()
        except:
            pass

In [ ]:
import itertools
import math
import os
import pickle
import warnings
from typing import Dict, List, Tuple

import pandas as pd
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
from MKLpy.algorithms import EasyMKL
from tqdm.notebook import tqdm

warnings.filterwarnings("ignore")

In [ ]:
plt.style.use("default")
plt.rcParams["font.size"] = 15

In [ ]:
set_latex()

## Load dataset

For downloading dataset, see https://github.com/LeoYu/neural-tangent-kernel-UCI

In [ ]:
DATA_DIR = os.path.join("./data/")

def get_datasize(dic: Dict) -> Tuple[int, int, int, int]:
    c = int(dic["n_clases="])
    d = int(dic["n_entradas="])
    n_train_val = int(dic["n_patrons1="])
    if "n_patrons2=" in dic:
        n_test = int(dic["n_patrons2="])
    else:
        n_test = 0
    n_tot = n_train_val + n_test
    return n_tot, n_train_val, n_test, d,  c


def load_data(dic: Dict) -> Tuple[np.array, np.array]:
    f = open(os.path.join(DATA_DIR, dic["dataset"], dic["fich1="]), "r").readlines()[1:]
    X = np.asarray(list(map(lambda x: list(map(float, x.split()[1:-1])), f)))
    y = np.asarray(list(map(lambda x: int(x.split()[-1]), f)))
    return X, y

In [ ]:
MAX_TOT = 1000
MAX_FEATURES = 10
MAX_CLASSES = 2

datasets = []

n_dataset = 0
for idx, dataset in enumerate(sorted(os.listdir(DATA_DIR))):
    if not os.path.isfile(os.path.join(DATA_DIR, dataset, f"{dataset}.txt")):
        continue

    if dataset == "tic-tac-toe":  # remove tic-tac-toe
        continue

    # load configuration
    dic = dict()
    dic["dataset"] = dataset

    for k, v in map(
        lambda x: x.split(),
        open(os.path.join(DATA_DIR, dataset, f"{dataset}.txt"), "r").readlines(),
    ):
        dic[k] = v

    # Check skip or not
    n_tot, n_train_val, n_test, n_feature, n_class = get_datasize(dic)
    if (
        (n_tot > MAX_TOT)
        or (n_test > 0)
        or (n_feature > MAX_FEATURES)
        or (n_class > MAX_CLASSES)
    ):
        continue
    else:
        print(f"-----{idx}, {dataset}, {n_tot}, {n_feature}, {n_class}-----")
        n_dataset += 1

    # load dataset
    X, y = load_data(dic)
    fold = list(
        map(
            lambda x: list(map(int, x.split())),
            open(
                os.path.join(DATA_DIR, dic["dataset"], "conxuntos_kfold.dat"), "r"
            ).readlines(),
        )
    )
    datasets.append({"X": X, "y": y, "name": dic["dataset"], "fold": fold})

## Kernels

In [ ]:
def calc_tau(alpha: float, S: np.array, diag_i: np.array, diag_j: np.array) -> np.array:
    tau = 1 / 4 + 1 / (2 * math.pi) * np.arcsin(
        ((alpha**2) * S)
        / (np.sqrt(((alpha**2) * diag_i + 0.5) * ((alpha**2) * diag_j + 0.5)))
    )
    return tau


def calc_tau_dot(
    alpha: float, S: np.array, diag_i: np.array, diag_j: np.array
) -> np.array:
    tau_dot = (
        (alpha**2)
        / (math.pi)
        * 1
        / np.sqrt(
            (2 * (alpha**2) * diag_i + 1) * (2 * (alpha**2) * diag_j + 1)
            - (4 * (alpha**4) * (S**2))
        )
    )
    return tau_dot

In [ ]:
def hard_kernel(X: np.array, alpha: float, beta: float, finetune: bool, rulelist: list):
    S_list = []
    tau_list = []
    tau_dot_list = []

    for feature_index in range(len(X[0])):
        S = np.outer(X[:, feature_index], X[:, feature_index].T) + beta**2
        S_all = np.matmul(X, X.T) + beta**2
        if finetune:
            S_list.append(S_all)
        else:
            S_list.append(S)

        _diag = [S[i, i] for i in range(len(S))]
        diag_i = np.array(_diag * len(_diag)).reshape(len(_diag), len(_diag))
        diag_j = diag_i.transpose()
        tau_list.append(calc_tau(alpha, S, diag_i, diag_j))
        tau_dot_list.append(calc_tau_dot(alpha, S, diag_i, diag_j))

    K = np.zeros((X.shape[0], X.shape[0]))

    H = np.zeros_like(S_list[0])
    for rules in tqdm(rulelist, leave=False):
        # Internal nodes
        for i, s in enumerate(rules):
            ts = rules[0:i] + rules[i + 1 :]
            _H_nodes = S_list[s] * tau_dot_list[s]
            for t in ts:
                _H_nodes *= tau_list[t]
            K += _H_nodes * (2 ** len(rules))
        _H_leaves = np.ones_like(K)

        # Leaves
        for tau in [tau_list[i] for i in rules]:
            _H_leaves *= tau
        K += _H_leaves * (2 ** len(rules))

    return K / len(rulelist)

## Experiment

In [ ]:
def extract_kernels(X, alpha, beta, degree):
    assert degree in (1, 2, 3)
    patterns = list(itertools.combinations(np.arange(X.shape[1]), 1))

    if degree >= 2:
        patterns.extend(list(itertools.combinations(np.arange(X.shape[1]), 2)))

    if degree >= 3:
        patterns.extend(list(itertools.combinations(np.arange(X.shape[1]), 3)))

    patterns = [list(l) for l in patterns]
    patterns = [[pattern] for pattern in patterns]

    kernels_aaa = []
    kernels_aai = []

    for pattern in tqdm(patterns, leave=False):
        kernels_aaa.append(
            hard_kernel(X, alpha=alpha, beta=beta, finetune=False, rulelist=pattern)
        )
        kernels_aai.append(
            hard_kernel(X, alpha=alpha, beta=beta, finetune=True, rulelist=pattern)
        )

    return kernels_aaa, kernels_aai, patterns

In [ ]:
def plot_weights(ker_matrix_aaa, ker_matrix_aai, patterns, name):
    x = range(len(ker_matrix_aaa.weights))
    plt.bar(x, ker_matrix_aaa.weights, alpha=0.5, label="AAA")
    plt.bar(x, ker_matrix_aai.weights, alpha=0.5, label="AAI")
    plt.xticks(
        x,
        [str(sorted(set(i[0]))).replace("[", "{").replace("]", "}") for i in patterns],
        rotation=75,
        fontsize=10
    )

    plt.grid(linestyle="dotted")
    plt.ylabel("Weight")
    x = range(len(ker_matrix_aai.weights))
    plt.grid(linestyle="dotted")
    plt.ylabel("Weight")
    plt.title(name)
    plt.legend(loc='upper left')
    plt.tight_layout()

In [ ]:
def kl_divergence(p, q):
    return np.sum(p * np.log(p / q))

In [ ]:
alpha = 2.0
beta = 0.5
degree = 2

kl_div_aaa = []
kl_div_aai = []

plt.figure(figsize=(20, 25))
for i, data in enumerate(datasets):
    X = data["X"]
    y = data["y"]
    name = data["name"]

    kernels_aaa, kernels_aai, patterns = extract_kernels(
        X, alpha=alpha, beta=beta, degree=degree
    )

    mkl = EasyMKL()
    ker_matrix_aaa = mkl.combine_kernels(kernels_aaa, y)
    ker_matrix_aai = mkl.combine_kernels(kernels_aai, y)

    kl_div_aaa.append(
        kl_divergence(
            np.array(ker_matrix_aaa.weights),
            np.ones_like(np.array(ker_matrix_aaa.weights))
            / len(np.array(ker_matrix_aaa.weights)),
        )
    )
    kl_div_aai.append(
        kl_divergence(
            np.array(ker_matrix_aai.weights),
            np.ones_like(np.array(ker_matrix_aai.weights))
            / len(np.array(ker_matrix_aai.weights)),
        )
    )

    plt.subplot(14, 1, i + 1)
    plot_weights(ker_matrix_aaa, ker_matrix_aai, patterns, name)
plt.savefig(f"./figures/mkl_weight.pdf", bbox_inches="tight", pad_inches=0.10)

In [ ]:
plt.figure(figsize=(6, 4))
plt.scatter(kl_div_aaa, kl_div_aai)
plt.plot([0, 0.8], [0, 0.8], linestyle="dotted", color="black")
plt.grid(linestyle="dotted")
plt.title("Kullback–Leibler divergence between obtained weights by MKL and uniform weights")
plt.xlabel("AAA")
plt.ylabel("AAI")
plt.savefig(f"./figures/mkl_kldiv.pdf", bbox_inches="tight", pad_inches=0.10)

## Generalization Performance

In [ ]:
def soft_kernel(X: np.array, depth: int, alpha: float, beta: float):
    K = np.zeros((depth, X.shape[0], X.shape[0]))
    S = np.matmul(X, X.T) + beta**2
    _diag = [S[i, i] for i in range(len(S))]
    diag_i = np.array(_diag * len(_diag)).reshape(len(_diag), len(_diag))
    diag_j = diag_i.transpose()

    tau = calc_tau(alpha, S, diag_i, diag_j)
    tau_dot = calc_tau_dot(alpha, S, diag_i, diag_j)

    for i, depth in enumerate((range(1, depth + 1, 1))):
        H = (2 * S * (2 ** (depth - 1)) * depth * tau_dot * tau ** (depth - 1)) + (
            (2**depth) * (tau**depth)
        )
        K[depth - 1] = H

    return K[::-1][0]

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


def svm(kernels, y, weights, reg, train_index, test_index):
    model = SVC(kernel="precomputed", C=1.0, probability=True)

    K = np.zeros_like(kernels[0])
    for j in range(len(weights)):
        K += kernels[j] * weights[j]

    K_train = K[train_index][:, train_index]
    K_test = K[test_index][:, train_index]

    y_train = y[train_index]
    y_test = y[test_index]

    model.fit(K_train, y_train)
    test_pred = model.predict(K_test)
    test_pred_proba = model.predict_proba(K_test)[:, 1]

    accuracy = accuracy_score(y_test, test_pred)
 
    return accuracy

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier


def rf_benchmark(
    X: np.array,
    y: np.array,
    train_index: list,
    test_index: list,
    max_depth: int,
    n_estimators: int,
    max_features: int,
) -> Tuple[float, List[float]]:
    model = RandomForestClassifier(
        max_depth=max_depth, n_estimators=n_estimators, max_features=max_features
    )
    model.fit(X[train_index], y[train_index])
    test_pred = model.predict(X[test_index])
    test_pred_proba = model.predict_proba(X[test_index])[:, 1]

    accuracy = accuracy_score(y[test_index], test_pred)

    return accuracy


def gbdt_benchmark(
    X: np.array,
    y: np.array,
    train_index: list,
    test_index: list,
    max_depth: int,
    n_estimators: int,
    max_features: int,
) -> Tuple[float, List[float]]:
    model = GradientBoostingClassifier(
        max_depth=max_depth, n_estimators=n_estimators, max_features=max_features
    )
    model.fit(X[train_index], y[train_index])
    test_pred = model.predict(X[test_index])
    test_pred_proba = model.predict_proba(X[test_index])[:, 1]

    accuracy = accuracy_score(y[test_index], test_pred)

    return accuracy

In [ ]:
def benchmark(kernels_aaa, kernels_aai,kernel_soft,  y, train_index, test_index, alpha, beta, repeat):
    acc_dict= {}
    
    acc_dict["alpha"] = alpha
    acc_dict["beta"] = beta
    acc_dict["repeat"] = repeat

    # AAA
    acc_dict["aaa_mkl"] = svm(kernels_aaa, y, np.array(ker_matrix_aaa.weights), 1.0, train_index, test_index)
    acc_dict["aaa_benchmark"] = svm(kernels_aaa, y, np.ones_like(ker_matrix_aaa.weights)/len(ker_matrix_aaa.weights), 1.0, train_index, test_index)

    # AAI
    acc_dict["aai_mkl"] = svm(kernels_aai, y, np.array(ker_matrix_aai.weights), 1.0, train_index, test_index)
    acc_dict["aai_benchmark"] = svm(kernels_aai, y, np.ones_like(ker_matrix_aai.weights)/len(ker_matrix_aaa.weights), 1.0, train_index, test_index)

    # Soft
    acc_dict["soft"] = svm([kernel_soft] * len(kernels_aaa), y, np.ones_like(ker_matrix_aaa.weights)/len(ker_matrix_aaa.weights), 1.0, train_index, test_index)

    max_features="sqrt"
    
    # RF
    acc_dict["rf3"] = rf_benchmark(X, y, train_index, test_index, max_depth=3, n_estimators=1000, max_features=max_features)
    acc_dict["rf5"] = rf_benchmark(X, y, train_index, test_index, max_depth=5, n_estimators=1000, max_features=max_features)
    acc_dict["rf7"] = rf_benchmark(X, y, train_index, test_index, max_depth=7, n_estimators=1000, max_features=max_features)
    acc_dict["rfmax"] = rf_benchmark(X, y, train_index, test_index, max_depth=None, n_estimators=1000, max_features=max_features)

    # GBDT
    acc_dict["gbdt3"] = gbdt_benchmark(X, y, train_index, test_index, max_depth=3, n_estimators=1000, max_features=max_features)
    acc_dict["gbdt5"] = gbdt_benchmark(X, y, train_index, test_index, max_depth=5, n_estimators=1000, max_features=max_features)
    acc_dict["gbdt7"] = gbdt_benchmark(X, y, train_index, test_index, max_depth=7, n_estimators=1000, max_features=max_features)
    acc_dict["gbdtmax"] = gbdt_benchmark(X, y, train_index, test_index, max_depth=None, n_estimators=1000, max_features=max_features)

    return acc_dict

In [ ]:
degree = 2

acc_dicts_multiple = []

if False:
    for i, data in enumerate(tqdm(datasets, leave=False)):
        X = data["X"]
        y = data["y"]
        fold = data["fold"]
        acc_dicts = []

        for alpha in tqdm([0.5, 1.0, 2.0, 4.0], leave=False):
            for beta in tqdm([0.1, 0.5, 1.0], leave=False):
                kernel_soft = soft_kernel(X, depth=degree, alpha=alpha, beta=beta)
                kernels_aaa, kernels_aai, patterns = extract_kernels(X, alpha=alpha, beta=beta, degree=degree)

                for repeat in tqdm(range(4), leave=False):
                    test_index, train_index = fold[repeat * 2], fold[repeat * 2 + 1]
                    assert len(test_index) > len(train_index)
                    mkl = EasyMKL()

                    train_kernels_aaa = [i[train_index][:, train_index] for i in kernels_aaa]
                    train_kernels_aai = [i[train_index][:, train_index] for i in kernels_aai]
                    ker_matrix_aaa = mkl.combine_kernels(train_kernels_aaa, y[train_index])
                    ker_matrix_aai = mkl.combine_kernels(train_kernels_aai, y[train_index])                    

                    acc_dict = benchmark(kernels_aaa, kernels_aai, kernel_soft, y, train_index, test_index, alpha, beta, repeat)

                    acc_dicts.append(acc_dict)
        acc_dicts_multiple.append(acc_dicts)

    with open('acc_dicts_multiple.pkl', 'wb') as file:
        pickle.dump(acc_dicts_multiple, file)

In [ ]:
with open('acc_dicts_multiple.pkl', 'rb') as file:
    acc_dicts_multiple = pickle.load(file)

In [ ]:
def plot_result(beta):
    plt.figure(figsize=(19,6))
    for i in range(len(acc_dicts_multiple)):
        plt.subplot(2,7,i+1)
        df = pd.DataFrame(acc_dicts_multiple[i])

        _df = df[df["beta"]==beta].groupby(by=["alpha", "beta"]).mean()[
            ["aaa_mkl", "aaa_benchmark", "aai_mkl", "aai_benchmark", "soft", "rf3", "rf5", "rf7", "gbdt3", "gbdt5", "gbdt7"]
        ].reset_index()

        x = range(4)

        _df["aaa_mkl"].plot(label="AAA (MKL)", color="red", linestyle="solid", marker="o")
        _df["aaa_benchmark"].plot(label="AAA (Benchmark)", color="red", linestyle="dotted", marker="v" )
        _df["aai_mkl"].plot(label="AAI (MKL)", color="blue", linestyle="solid", marker="o")
        _df["aai_benchmark"].plot(label="AAI (Benchmark)", color="blue", linestyle="dotted", marker="v")
        _df["soft"].plot(label="Oblique", color="black", marker="s")

        rf3_mean = df.groupby(by=["alpha", "beta"]).mean().mean()["rf3"]
        rf3_std  = df.groupby(by=["alpha", "beta"]).mean().std()["rf3"]
        rf5_mean = df.groupby(by=["alpha", "beta"]).mean().mean()["rf5"]
        rf5_std  = df.groupby(by=["alpha", "beta"]).mean().std()["rf5"]
        rf7_mean = df.groupby(by=["alpha", "beta"]).mean().mean()["rf7"]
        rf7_std  = df.groupby(by=["alpha", "beta"]).mean().std()["rf7"]
        rfmax_mean = df.groupby(by=["alpha", "beta"]).mean().mean()["rfmax"]
        rfmax_std  = df.groupby(by=["alpha", "beta"]).mean().std()["rfmax"]

        gbdt3_mean = df.groupby(by=["alpha", "beta"]).mean().mean()["gbdt3"]
        gbdt3_std  = df.groupby(by=["alpha", "beta"]).mean().std()["gbdt3"]
        gbdt5_mean = df.groupby(by=["alpha", "beta"]).mean().mean()["gbdt5"]
        gbdt5_std  = df.groupby(by=["alpha", "beta"]).mean().std()["gbdt5"]
        gbdt7_mean = df.groupby(by=["alpha", "beta"]).mean().mean()["gbdt7"]
        gbdt7_std  = df.groupby(by=["alpha", "beta"]).mean().std()["gbdt7"]
        gbdtmax_mean = df.groupby(by=["alpha", "beta"]).mean().mean()["gbdtmax"]
        gbdtmax_std  = df.groupby(by=["alpha", "beta"]).mean().std()["gbdtmax"]

        plt.plot(x, [rf3_mean]*len(x), color="green", linestyle=(3, (6, 6)), alpha=0.7, label="Random Forest", linewidth=1.0)
        plt.plot(x, [rfmax_mean]*len(x), color="green", linestyle=(3, (1, 1)), alpha=0.7, label="Random Forest", linewidth=1.0)
        plt.plot(x, [gbdt3_mean]*len(x), color="orange", linestyle=(0, (6, 6)), alpha=0.7, label="Gradient Boosting", linewidth=1.0)
        plt.plot(x, [gbdtmax_mean]*len(x), color="orange", linestyle=(0, (1, 1)), alpha=0.7, label="Gradient Boosting", linewidth=1.0)

        plt.fill_between(x, rf3_mean-rf3_std, rf3_mean+rf3_std, color='green', alpha=0.1)
        plt.fill_between(x, rfmax_mean-rfmax_std, rfmax_mean+rfmax_std, color='green', alpha=0.1)
        plt.fill_between(x, gbdt3_mean-gbdt3_std, gbdt3_mean+gbdt3_std, color='orange', alpha=0.1)
        plt.fill_between(x, gbdtmax_mean-gbdtmax_std, gbdtmax_mean+gbdtmax_std, color='orange', alpha=0.1)

        plt.title(datasets[i]["name"])
        if i>=7:
            plt.xlabel("$\\alpha$")
            plt.xticks([0, 1, 2, 3], [0.5, 1.0, 2.0, 4.0])
        else:
            plt.xticks([0, 1, 2, 3], [])
        if i%7 ==0:
            plt.ylabel("Accuracy")
        plt.grid(linestyle="dotted")

    plt.figlegend(labels=[
        "AAA (MKL)", 
        "AAA (Benchmark)", 
        "AAI (MKL)", 
        "AAI (Benchmark)", 
        "Oblique", 
        "RF (max_depth=3)",
        "RF (max_depth=None)", 
        "GBDT (max_depth=3)",  
        "GBDT (max_depth=None)"
    ],
                  loc="lower center", 
                  ncol=5,
                  bbox_to_anchor=(0.515, -0.15))

    plt.tight_layout()
    plt.suptitle(f"$\\beta$={beta}", y=1.05)
    plt.savefig(f"./figures/multidata_metrics_{beta}.pdf", bbox_inches="tight", pad_inches=0.10)

In [ ]:
plot_result(beta=0.1)

In [ ]:
plot_result(beta=0.5)

In [ ]:
plot_result(beta=1.0)